In [12]:
import sys
import os
import glob
import re

import pandas as pd
import numpy as np
from tqdm import tqdm

In [13]:
df = pd.read_csv('../processed_data/train_v2.csv')
df.head(3)

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,enviroment,structure,contact_period,max_floor,room_num,L,D,K,S,north
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,7,2年間,12,1,0,0,1,0,0
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,...,【スーパー】 1283m,2,2年間,10,1,0,0,0,0,0
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,102,南,22.05,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,...,【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,7,2年間,15,1,0,0,1,0,0


### structure の前処理

- コストの面からみた順序尺度が存在すると仮定

In [14]:
df['structure'].value_counts()

7    17505
1     5648
2     3726
8     2429
4     1812
6      297
5       46
3        4
0        3
Name: structure, dtype: int64

In [15]:
df['structure'].unique()

array([7, 2, 1, 8, 4, 6, 5, 3, 0])

In [16]:
df['structure'].isnull().sum()

0

In [17]:
df.groupby(by='structure').describe()['y']

,count,mean,std,min,25%,50%,75%,max
structure,,,,,,,,
0,3.0,60000.000000,15000.000000,45000.0,52500.0,60000.0,67500.0,75000.0
1,5648.0,83772.665722,56318.694811,25000.0,58000.0,69000.0,90000.0,1450000.0
2,3726.0,91993.971014,51622.538387,35000.0,67000.0,80000.0,102000.0,1390000.0
3,4.0,93000.000000,41044.691090,55000.0,69250.0,83500.0,107250.0,150000.0
4,1812.0,94458.160044,34477.836339,40000.0,70000.0,87000.0,113075.0,450000.0
5,46.0,106195.652174,21821.110640,69000.0,89250.0,102500.0,117250.0,159000.0
6,297.0,107103.703704,33898.290135,48000.0,81000.0,102000.0,127000.0,255000.0
7,17505.0,134366.134704,78147.007990,32000.0,92000.0,115000.0,155000.0,2500000.0
8,2429.0,142525.408399,85043.488185,42000.0,91000.0,120000.0,165000.0,1120000.0


- SRC, RC, ALC, PC, 軽量鉄骨,　HPC,　鉄骨造,その他・木造,ブロックの順に,8,7,6,5,4,3,2,1,0に置き換え

In [8]:
tmp = df['structure'].map({
    'SRC（鉄骨鉄筋コンクリート）': 8,
    'RC（鉄筋コンクリート）': 7,
    'ALC（軽量気泡コンクリート）': 6,
    'PC（プレキャスト・コンクリート（鉄筋コンクリート））': 5,
    '軽量鉄骨': 4,
    'HPC（プレキャスト・コンクリート（重量鉄骨））': 3,
    '鉄骨造':  2,
    'その他': 1,
    '木造': 1,
    '鉄骨ブロック': 0,
    'ブロック': 0
})

In [18]:
tmp.isnull().sum()

31470

In [10]:
df['structure'] = tmp

In [19]:
df

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,enviroment,structure,contact_period,max_floor,room_num,L,D,K,S,north
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,117,南東,20.01,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,7,2年間,12,1,0,0,1,0,0
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,538,NaN,16.50,5,専用トイレ／\tシャワー／\t温水洗浄便座,...,【スーパー】 1283m,2,2年間,10,1,0,0,0,0,0
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,102,南,22.05,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,...,【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,7,2年間,15,1,0,0,1,0,0
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,352,南,60.48,3,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,...,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,7,2年間\t※この物件は\t定期借家\tです。,4,2,1,1,1,0,0
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,379,南,39.66,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,...,【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,1,2年間,2,2,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31465,31466,80000,東京都板橋区蓮根２丁目,都営三田線\t蓮根駅\t徒歩7分\t\t都営三田線\t西台駅\t徒歩10分\t\t都営三田線...,2DK,360,南,37.90,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t洗面台独立,...,NaN,2,2年間,3,2,0,1,1,0,0
31466,31467,54500,東京都世田谷区太子堂５丁目17-1,東急田園都市線\t三軒茶屋駅\t徒歩6分\t\t東急世田谷線\t西太子堂駅\t徒歩4分\t\...,1R,475,西,12.04,4,専用バス／\t専用トイレ／\tシャワー,...,【スーパー】 458m\t【スーパー】 540m\t【コンビニ】 131m\t【コンビニ】 ...,7,2年間,4,1,0,0,0,0,0
31467,31468,125000,東京都江東区南砂４丁目,東西線\t南砂町駅\t徒歩5分\t\t都営新宿線\t大島(東京都)駅\t徒歩26分\t\t東...,3DK,550,南,60.00,3,バス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座,...,【コンビニ】 302m\t【コンビニ】 394m\t【コンビニ】 452m\t【コンビニ】 ...,2,2年間,5,3,0,1,1,0,0
31468,31469,98000,東京都中野区中野２丁目,中央線（快速）\t中野(東京都)駅\t徒歩4分\t\t丸ノ内線(池袋－荻窪)\t新中野駅\t...,1DK,132,南,29.59,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t洗面台独立,...,【スーパー】 485m\t【スーパー】 1051m\t【コンビニ】 476m\t【コンビニ】...,1,2年間,2,1,0,1,1,0,0


In [20]:
df.to_csv('../processed_data/train_v3.csv',index=False)

In [43]:
test = pd.read_csv('../data/test.csv')
test.isnull().sum()

id           0
所在地          0
アクセス         0
間取り          0
築年数          0
方角        2808
面積           0
所在階          1
バス・トイレ     320
キッチン       635
放送・通信     3454
室内設備        73
駐車場       5356
周辺環境      9241
建物構造         0
契約期間      6981
dtype: int64

In [45]:
pd.read_csv('../hoge.csv')['structure'].head()

0    7
1    2
2    7
3    7
4    1
Name: structure, dtype: int64

In [46]:
df['location'].unique()

array(['東京都北区滝野川３丁目', '東京都中央区月島３丁目', '東京都渋谷区笹塚２丁目', ...,
       '東京都足立区堀之内２丁目13-3', '東京都千代田区神田三崎町２丁目１０－１０', '東京都世田谷区太子堂５丁目17-1'],
      dtype=object)

In [47]:
df.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contact_period', 'max_floor',
       'room_num', 'L', 'D', 'K', 'S', 'north'],
      dtype='object')

In [56]:
df['access'].apply(lambda x: x.split('\t'))

['都営三田線',
 '西巣鴨駅',
 '徒歩4分',
 '',
 '埼京線',
 '板橋駅',
 '徒歩14分',
 '',
 '都電荒川線',
 '西ケ原四丁目駅',
 '徒歩7分']

In [59]:
df['access'].apply(lambda x: x.split('\t'))

0        [都営三田線, 西巣鴨駅, 徒歩4分, , 埼京線, 板橋駅, 徒歩14分, , 都電荒川線...
1        [都営大江戸線, 勝どき駅, 徒歩5分, , 有楽町線, 月島駅, 徒歩9分, , 日比谷線...
2        [京王線, 笹塚駅, 徒歩6分, , 京王線, 代田橋駅, 徒歩7分, , 京王線, 明大前...
3        [総武線・中央線（各停）, 高円寺駅, 徒歩9分, , 丸ノ内線(池袋－荻窪), 新高円寺駅...
4        [京成金町線, 京成金町駅, 徒歩5分, , 常磐線, 金町(東京都)駅, 徒歩7分, , ...
                               ...                        
31465    [都営三田線, 蓮根駅, 徒歩7分, , 都営三田線, 西台駅, 徒歩10分, , 都営三田...
31466    [東急田園都市線, 三軒茶屋駅, 徒歩6分, , 東急世田谷線, 西太子堂駅, 徒歩4分, ...
31467    [東西線, 南砂町駅, 徒歩5分, , 都営新宿線, 大島(東京都)駅, 徒歩26分, , ...
31468    [中央線（快速）, 中野(東京都)駅, 徒歩4分, , 丸ノ内線(池袋－荻窪), 新中野駅,...
31469    [有楽町線, 千川駅, 徒歩5分, , 有楽町線, 要町駅, 徒歩13分, , 西武池袋線,...
Name: access, Length: 31470, dtype: object